In [10]:
from datetime import datetime
import MetaTrader5 as mt5
import time
import telepot
import pytz
import numpy as np

# importamos o módulo pandas para exibir os dados recebidos na forma de uma tabela
import pandas as pd
pd.set_option('display.max_columns', 500) # número de colunas
pd.set_option('display.width', 1500)      # largura máxima da tabela
pd.options.mode.chained_assignment = None  # default='warn'

#if not mt5.initialize(login=1092947504, server="ClearInvestimentos-DEMO", password="Joh0516"):
if not mt5.initialize(login=4999473749, server="MetaQuotes-Demo", password="elf4lnbx"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()
    

symbol = 'EURUSD'
#symbol = 'GBPUSD'
#symbol = 'USDJPY'

print(symbol)
rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M15, 0, 100)
rates_frame = pd.DataFrame(rates)
rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
resumo = rates_frame[['time','open','high','low','close','tick_volume']]
adxTab = resumo

def get_adx(high, low, close, lookback):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()
    return plus_di, minus_di, adx_smooth

adxTab['plus_di'] = pd.DataFrame(get_adx(adxTab['high'], adxTab['low'], adxTab['close'], 9)[0]).rename(columns = {0:'plus_di'})
adxTab['minus_di'] = pd.DataFrame(get_adx(adxTab['high'], adxTab['low'], adxTab['close'], 9)[1]).rename(columns = {0:'minus_di'})
adxTab['adx'] = pd.DataFrame(get_adx(adxTab['high'], adxTab['low'], adxTab['close'], 9)[2]).rename(columns = {0:'adx'})
adxTab = adxTab.dropna()
#aapl

resumoADX = aapl[['time','plus_di','minus_di','adx']]
display(resumoADX.tail(1))

EURUSD


,time,plus_di,minus_di,adx
99,2022-04-05 21:15:00,14.229142,50.632756,53.020126
